In [1]:
"""
Extract Active Fire Detections (AFDs) from MODIS and VIIRS within "aspen fires"

Author: maxwell.cook@colorado.edu
"""

import os, time, glob, sys
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import timedelta

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

datamod = os.path.join(projdir,'data/spatial/mod/FIRMS/')

print("Ready to go !")

Ready to go !


In [2]:
# Load the fire data (aspen fires).

In [3]:
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen.gpkg')
fires = gpd.read_file(fp)
fires.columns

Index(['NIFC_ID', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES', 'pct_cover',
       'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'geometry'],
      dtype='object')

In [4]:
# Create a fire perimeter buffer to extract FRP observations within 1km of bounds.
bounds = create_bounds(fires, buffer=1000, method='convex_hull')
bounds = bounds.set_crs(proj, allow_override=True) # ensure correct crs

# Tidy the columns
bounds = bounds[['NIFC_ID','NIFC_NAME','START_YEAR','DISCOVERY_DATE','WF_CESSATION_DATE','na_l3name','geometry']] 
bounds['START_YEAR'] = bounds['START_YEAR'].astype(int)
date_cols = ['DISCOVERY_DATE', 'WF_CESSATION_DATE']
for col in date_cols:
    bounds[col] = pd.to_datetime(bounds[col], errors='coerce')
bounds.head()

,NIFC_ID,NIFC_NAME,START_YEAR,DISCOVERY_DATE,WF_CESSATION_DATE,na_l3name,geometry
0,6,DEVIL CREEK,2018,2018-07-19 15:46:00,2018-07-21 15:00:00,Southern Rockies,"POLYGON ((-981645.032 1644521.242, -981698.906..."
1,8,577,2019,2019-07-28 14:22:00,2019-08-18 16:00:00,Southern Rockies,"POLYGON ((-1003680.565 1627021.419, -1003775.4..."
2,14,416,2018,2018-06-01 11:02:00,2018-07-03 18:00:00,Southern Rockies,"POLYGON ((-1038870.616 1657765.574, -1038969.6..."
3,16,NEBO,2020,2020-10-13 19:07:00,2020-10-15 17:00:00,Southern Rockies,"POLYGON ((-1002711.210 1682990.209, -1002813.6..."
4,23,LOADING PEN,2020,2020-06-13 21:42:00,2020-06-18 17:45:00,Southern Rockies,"POLYGON ((-1071537.321 1681471.519, -1071606.3..."


In [5]:
bounds.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

### Load the Active Fire Detections (AFD) [2018-2023]
AFDs from MODIS Collection 6.1 (1km) and the Suomi National Polar-Orbiting Partnership (VIIRS S-NPP 375m) were obtained from the NASA FIRMS (https://firms.modaps.eosdis.nasa.gov/download/) between 2018-2023 in the western US. The S-NPP AFDs include an archive shapefile and a NRT shapefile for Sept 2022-->

These files represent non-geolocated active fire pixel (lat/lon of fire detection pixels). 

In [6]:
# Gather the archive and NRT S-NPP AFDs.
snpp_fp = os.path.join(projdir,'data/spatial/raw/FIRMS/y2018_to_2023/DL_FIRE_SV-C2_476784/')
vects = glob.glob(snpp_fp+"*.shp")
print([os.path.basename(v) for v in vects])

['fire_nrt_SV-C2_476784.shp', 'fire_archive_SV-C2_476784.shp']


In [7]:
# Merge the NRT and archive vintages
archive = gpd.read_file([v for v in vects if "archive" in v][0]).to_crs(proj)
nrt = gpd.read_file([v for v in vects if "nrt" in v][0]).to_crs(proj)
snpp = pd.concat([archive, nrt], ignore_index=True)
snpp.head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,36.622398,-119.958420,319.25,0.45,0.63,2018-01-01,0854,N,VIIRS,n,1,276.46,1.93,N,0.0,POINT (-2100437.814 1774627.392)
1,48.166241,-102.698212,323.92,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,244.01,3.01,N,0.0,POINT (-502299.189 2814197.682)
2,48.152390,-102.697792,325.34,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,247.16,2.07,N,0.0,POINT (-502375.241 2812672.753)
3,47.879730,-102.706543,320.74,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,245.34,3.00,N,0.0,POINT (-505150.481 2782728.231)
4,44.249660,-104.516678,327.49,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,251.79,5.84,N,0.0,POINT (-677212.174 2392519.636)


In [8]:
# Set up a dictionary to store both MODIS and VIIRS AFDs
modis_fp = os.path.join(projdir,'data/spatial/raw/FIRMS/y2018_to_2023/DL_FIRE_M-C61_476781/')

# Store these in a dictionary
afds = {
    "MOD61": gpd.read_file(modis_fp).to_crs(proj),
    "SNPP": snpp
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,0608,Terra,MODIS,85,6.03,269.1,21.9,N,0,POINT (-1819292.701 2598646.798)
1,32.1679,-107.7655,322.3,1.1,1.0,2018-01-01,1809,Terra,MODIS,77,6.03,289.1,19.0,D,0,POINT (-1101321.656 1078752.715)
2,32.1643,-107.7433,318.4,1.1,1.0,2018-01-01,1809,Terra,MODIS,74,6.03,287.9,14.9,D,0,POINT (-1099303.652 1078097.526)
3,31.7188,-102.0085,303.8,1.4,1.2,2018-01-01,1944,Aqua,MODIS,57,6.03,279.4,12.6,D,0,POINT (-566656.554 978378.293)
4,38.8238,-122.6829,302.0,2.7,1.6,2018-01-01,1946,Terra,MODIS,50,6.03,280.2,26.9,D,0,POINT (-2264952.497 2075236.763)


In [9]:
afds['MOD61'].crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
# Extract the AFDs within fire bounds.

In [11]:
afd_events = {} # to store the filtered data
duplicate_list = [] # to store duplicate obs.

# Loop through sensors (MODIS and VIIRS)
for satellite, afd in afds.items():
    print(f"Processing: {satellite}\n")

    afd = afd.reset_index(drop=True)
    afd['afdID'] = afd.index # add a unique ID

    ####################################
    # Remove low confidence observations
    try:
        afd = afd[afd['CONFIDENCE'] != 'l']
    except KeyError as e:
        print(f"KeyError: {e}")

    ############################
    # Extract within fire bounds
    if afd.crs != bounds.crs:
        afd = afd.to_crs(bounds.crs)  # ensure the crs matches

    afd_ = gpd.sjoin(afd, bounds, how='inner', predicate='within')
    afd_.drop(columns=['index_right'], inplace=True)

    ####################################
    # Tidy the date columns for matching

    afd_f = afd_.copy()
    
    afd_f['ACQ_DATE'] = pd.to_datetime(afd_['ACQ_DATE'])
    afd_f['ACQ_DATETIME'] = afd_f.apply(lambda row: convert_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)
    afd_f['ACQ_MONTH'] = afd_f['ACQ_DATE'].dt.month.astype(int)
    afd_f['ACQ_YEAR'] = afd_f['ACQ_DATE'].dt.year.astype(int)
    
    # Filter for matching dates
    afd_f = afd_f[
        (afd_f['ACQ_DATE'] >= afd_f['DISCOVERY_DATE'] - timedelta(days=14)) &
        (afd_f['ACQ_DATE'] <= afd_f['WF_CESSATION_DATE'] + timedelta(days=14))
    ]

    #####################
    # Drop any duplicates
    duplicates = afd_f[afd_f.duplicated(subset='afdID', keep=False)]
    if len(duplicates) > 0:
        print(f"Be aware ! Duplicates found ! {len(duplicates)} to be exact ...")
        duplicate_list.append(duplicates)
        afd_f = afd_f.drop_duplicates(subset='afdID', keep='first')

    #####################
    # Add observation counts as an attribute
    counts = afd_f.groupby(['NIFC_ID']).size().reset_index(name='obs_count')
    afd_f = pd.merge(afd_f, counts, left_on='NIFC_ID', right_on='NIFC_ID', how='left')

    ######################
    print(f"\tThere are [{len(afd_f)}] detections within fire bounds.")

    # Overwrite the dictionary items
    afd_events[satellite] = afd_f

    # Save the AFD observation in fire bounds
    out_fp = os.path.join(datamod,f'{satellite}/{satellite}-afd_aspen-fires_2018_to_2023.gpkg')
    afd_f.to_file(out_fp)
    print(f'Saved to {out_fp}\n')
    
    del afd, afd_, afd_f, duplicates
    gc.collect()

# Export duplicates if they exist
if duplicate_list:
    duplicates_gdf = pd.concat(duplicate_list, ignore_index=True)
    out_fp = os.path.join(datamod,'afd_duplicate_detections.gpkg')
    duplicates_gdf.to_file(out_fp)
    print(f"Saved duplicates to {out_fp}")
    
print("\nProcessing complete !")

Processing: MOD61

Be aware ! Duplicates found ! 1832 to be exact ...
	There are [62170] detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/FIRMS/MOD61/MOD61-afd_aspen-fires_2018_to_2023.gpkg

Processing: SNPP

Be aware ! Duplicates found ! 8064 to be exact ...
	There are [261644] detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/FIRMS/SNPP/SNPP-afd_aspen-fires_2018_to_2023.gpkg

Saved duplicates to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/FIRMS/afd_duplicate_detections.gpkg

Processing complete !


In [12]:
afd_events['MOD61'].columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'afdID', 'NIFC_ID',
       'NIFC_NAME', 'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE',
       'na_l3name', 'ACQ_DATETIME', 'ACQ_MONTH', 'ACQ_YEAR', 'obs_count'],
      dtype='object')

In [13]:
# Grab a count and first/last dates for VIIRS obs.

In [14]:
viirs = afd_events['SNPP']
summary = (
    viirs.groupby(['NIFC_ID'])
        .agg(
            first_obs_date=('ACQ_DATE', 'min'),  # First observation date for each fire
            last_obs_date=('ACQ_DATE', 'max'),   # Last observation date for each fire
            obs_count=('ACQ_DATE', 'size')       # Count of observations for each fire
        ).reset_index()  # Reset index to make 'NIFC_ID' a column in the resulting DataFrame
)
summary['obs_count'] = summary['obs_count'].fillna(0).astype(int) # fill NaN as 0 obs.
summary.head()

,NIFC_ID,first_obs_date,last_obs_date,obs_count
0,100385,2021-08-13,2021-08-15,39
1,100719,2022-09-05,2022-09-07,6
2,100763,2022-10-05,2022-10-20,16
3,101285,2023-09-16,2023-09-23,103
4,101286,2023-09-03,2023-09-15,7


In [15]:
# Join this information to the NIFC fire perimeters
fires = pd.merge(fires, summary, left_on='NIFC_ID', right_on='NIFC_ID', how='left')
fires.columns

Index(['NIFC_ID', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES', 'pct_cover',
       'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'geometry', 'first_obs_date',
       'last_obs_date', 'obs_count'],
      dtype='object')

In [16]:
# Save out (overwrite).
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen-obs.gpkg')
fires.to_file(fp)
print(f"Saved to: {fp}")

Saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen-obs.gpkg


In [17]:
# Create the AFD "detection buffer" for both MODIS and VIIRS

In [18]:
afd_events_plots = {} # to store the buffered observations
for satellite, afd in afd_events.items():
    print(f"Creating buffer for {satellite} observations.")

    afd_ = afd.copy()
    if satellite == 'MOD61':
        afd_.geometry = afd_.geometry.buffer(500, cap_style=3) # square buffer half of pixel size
    elif satellite == 'SNPP':
        afd_.geometry = afd_.geometry.buffer(187.5, cap_style=3) # square buffer half of pixel size

    afd_events_plots[satellite] = afd_ # add to the new dictionary

    # Save out
    out_fp = os.path.join(datamod,f'{satellite}/{satellite}-afd_aspen-fires_2018_to_2023_pix.gpkg')
    afd_.to_file(out_fp)
    print(f'Saved to {out_fp}\n')

    del afd_
    gc.collect()

Creating buffer for MOD61 observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/FIRMS/MOD61/MOD61-afd_aspen-fires_2018_to_2023_pix.gpkg

Creating buffer for SNPP observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/FIRMS/SNPP/SNPP-afd_aspen-fires_2018_to_2023_pix.gpkg



In [19]:
print(afd_events_plots['MOD61'].columns)
print("\n~~~~~~~~~\n")
print(afd_events_plots['SNPP'].columns)

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'afdID', 'NIFC_ID',
       'NIFC_NAME', 'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE',
       'na_l3name', 'ACQ_DATETIME', 'ACQ_MONTH', 'ACQ_YEAR', 'obs_count'],
      dtype='object')

~~~~~~~~~

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'afdID', 'NIFC_ID',
       'NIFC_NAME', 'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE',
       'na_l3name', 'ACQ_DATETIME', 'ACQ_MONTH', 'ACQ_YEAR', 'obs_count'],
      dtype='object')


In [20]:
# Combine the MODIS and VIIRS buffers into a single GeoDataFrame
afd_events_plots_c = pd.concat([afd_events_plots['MOD61'], afd_events_plots['SNPP']], ignore_index=True)
afd_events_plots_c['afdID'] = afd_events_plots_c['INSTRUMENT'].astype(str) + afd_events_plots_c['afdID'].astype(str)
afd_events_plots_c[['NIFC_ID','NIFC_NAME','afdID']].head()

,NIFC_ID,NIFC_NAME,afdID
0,11299,RATTLESNAKE,MODIS3703
1,11299,RATTLESNAKE,MODIS3704
2,11299,RATTLESNAKE,MODIS3740
3,11299,RATTLESNAKE,MODIS3742
4,11299,RATTLESNAKE,MODIS3743


In [26]:
# Only keep events with > X observations from either instrument
print(f"Number of AFDs before filtering: {len(afd_events_plots_c)}\nAcross {len(afd_events_plots_c['NIFC_ID'].unique())} unique fires.")
n_obs = 10 # greater than N observations (from VIIRS)
afd_counts = afd_events_plots_c.groupby('NIFC_ID')['INSTRUMENT'].count()
print(afd_counts.describe())
viirs_counts = afd_counts[afd_counts['INSTRUMENT'] == 'VIIRS']
ids = afd_counts[afd_counts["counts"] >= 10] # list of IDs with > 10 obs.
ids = ids['NIFC_ID'].unique().tolist()
# Keep only fires with greater than N obs.
afd_events_plots_c_ = afd_events_plots_c[afd_events_plots_c['NIFC_ID'].isin(ids)]
print(f"Number of AFDs after filtering: {len(afd_events_plots_c_)}\nAcross {len(afd_events_plots_c_['NIFC_ID'].unique())} unique fires.")

Number of AFDs before filtering: 323814
Across 459 unique fires.
count      459.000000
mean       705.477124
std       1715.081993
min          1.000000
25%         16.000000
50%        102.000000
75%        496.500000
max      15962.000000
Name: INSTRUMENT, dtype: float64


KeyError: 'INSTRUMENT'

In [ ]:
# Export to file.
out_fp = os.path.join(datamod,f'combined-afd_aspen-fires_2018_to_2023_pix.gpkg')
afd_events_plots_c_.to_file(out_fp)
print(f'Saved to {out_fp}\n')

In [ ]:
print(len(afd_events_plots_c_['NIFC_ID'].unique()))

In [ ]:
# Gather bounds for individual fires/AFDs for both instruments.

In [ ]:
buffer = 1000 # meters
# Group by fire ID
grouped = afd_events_plots_c.groupby('NIFC_ID')
bounds_gdf = gpd.GeoDataFrame(columns=['NIFC_ID', 'geometry'], crs=afd.crs)
# Iterate over each fire group
for fire_id, group in grouped:
    # Calculate the total bounds (minx, miny, maxx, maxy) for all geometries in the group
    bounds = box(*group.total_bounds)
    # Apply buffer if specified
    if buffer is not None:
        bounds = bounds.buffer(buffer)
    # Create a GeoDataFrame for this fire with the bounding box, id, and ig_year
    fire_gdf = gpd.GeoDataFrame({'NIFC_ID': [fire_id], 'geometry': [bounds]}, crs=afd.crs)
    # Append the result to the GeoDataFrame
    bounds_gdf = pd.concat([bounds_gdf, fire_gdf], ignore_index=True)

# Dissolve by NIFC_ID
afd_fire_bounds = bounds_gdf.dissolve(by='NIFC_ID')
afd_fire_bounds = pd.merge(afd_fire_bounds, aspen_events.drop(columns=['geometry']), left_on='NIFC_ID', right_on='NIFC_ID', how='left')
print(len(afd_fire_bounds['NIFC_ID'].unique()))
print(afd_fire_bounds.columns)

In [ ]:
# Make some plots.

In [ ]:
afd_events_plots_c['INSTRUMENT'].unique()

In [ ]:
# Day/Night FRP

plt.figure(figsize=(8, 6))

df = afd_events_plots_c.copy() # make a copy for plotting

# Filter day and night observations for MODIS and VIIRS
modis_day = df[(df['DAYNIGHT'] == 'D') & (df['INSTRUMENT'] == 'MODIS')]['FRP']
modis_night = df[(df['DAYNIGHT'] == 'N') & (df['INSTRUMENT'] == 'MODIS')]['FRP']
viirs_day = df[(df['DAYNIGHT'] == 'D') & (df['INSTRUMENT'] == 'VIIRS')]['FRP']
viirs_night = df[(df['DAYNIGHT'] == 'N') & (df['INSTRUMENT'] == 'VIIRS')]['FRP']

# Log scale for each subset
modis_day_log = np.log(modis_day[modis_day > 0])
modis_night_log = np.log(modis_night[modis_night > 0])
viirs_day_log = np.log(viirs_day[viirs_day > 0])
viirs_night_log = np.log(viirs_night[viirs_night > 0])

# Show the distribution of FRP values for day and night observations
plt.figure(figsize=(6, 4))

# Plotting the KDE
sns.kdeplot(modis_day_log, fill=True, color="orange", label="MODIS Day", alpha=0.6)
sns.kdeplot(modis_night_log, fill=True, color="orange", linestyle='--', label="MODIS Night", alpha=0.6)
sns.kdeplot(viirs_day_log, fill=True, color="blue", label="VIIRS Day", alpha=0.6)
sns.kdeplot(viirs_night_log, fill=True, color="blue", linestyle='--', label="VIIRS Night", alpha=0.6)

# Adding labels and title
plt.xlabel('Fire Radiative Power (FRP)')
plt.ylabel('Density')
plt.legend()

plt.savefig(os.path.join(maindir,'aspen-fire/Aim2/figures/FigX_frp_density_day-night.png'), dpi=300, bbox_inches='tight')

plt.show()

In [26]:
# Join to state boundaries, assign "Start_Day" and "End_Day" from Parks et al. (2019)

In [27]:
states = os.path.join(maindir, 'data/boundaries/political/TIGER/tl19_us_states_w_ak_lambert.gpkg')
states = gpd.read_file(states)
states = states.to_crs(proj)
states.columns

Index(['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME',
       'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry'],
      dtype='object')

In [28]:
centroid = afd_fire_bounds.copy()
centroid['geometry'] = centroid.geometry.centroid
fire_state = gpd.overlay(centroid[['NIFC_ID','geometry']], states[['STUSPS','geometry']], how='intersection')
fire_state = fire_state[['NIFC_ID','STUSPS']]
afd_fire_bounds_ = pd.merge(afd_fire_bounds, fire_state, on='NIFC_ID', how='left')
afd_fire_bounds_.columns

Index(['NIFC_ID', 'geometry', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES',
       'pct_aspen', 'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'WF_CESSATION_DATE',
       'WF_CESSATION_DOY', 'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL',
       'STR_THREATENED_MAX', 'EVACUATION_REPORTED', 'PEAK_EVACUATIONS',
       'WF_PEAK_AERIAL', 'WF_PEAK_PERSONNEL', 'na_l3name', 'STUSPS'],
      dtype='object')

In [29]:
len(afd_fire_bounds_[afd_fire_bounds_['na_l3name'] == 'Southern Rockies'])

61

In [30]:
# Assign the Parks et al. (2019) start and end days
special_case = ['Arizona', 'New Mexico']
afd_fire_bounds_['Start_Day'] = None
afd_fire_bounds_['End_Day'] = None    

afd_fire_bounds_.loc[afd_fire_bounds_['STUSPS'].isin(special_case), ['Start_Day', 'End_Day']] = (91, 181)
afd_fire_bounds_.loc[~afd_fire_bounds_['STUSPS'].isin(special_case), ['Start_Day', 'End_Day']] = (152, 258)

afd_fire_bounds_ = afd_fire_bounds_[['NIFC_ID','NIFC_NAME','DISCOVERY_DOY','WF_CESSATION_DOY','START_YEAR','Start_Day','End_Day','geometry']]
afd_fire_bounds_ = afd_fire_bounds_.rename(columns={
    'NIFC_ID': 'Fire_ID', 
    'NIFC_NAME': 'Fire_Name',
    'START_YEAR': 'Fire_Year'
})
afd_fire_bounds_.head()

,Fire_ID,Fire_Name,DISCOVERY_DOY,WF_CESSATION_DOY,Fire_Year,Start_Day,End_Day,geometry
0,100763,PARK CREEK,272,286,2022,152,258,"POLYGON ((-1302777.051 2935597.721, -1304470.8..."
1,1055,RATTLESNAKE,232,236,2020,152,258,"POLYGON ((-1662672.883 2765896.178, -1665553.8..."
2,1070,GREEN RIDGE,188,250,2021,152,258,"POLYGON ((-1648322.560 2755617.377, -1673173.6..."
3,12229,SCHAEFFER,251,272,2019,152,258,"POLYGON ((-1978906.428 1675019.040, -1982045.2..."
4,13137,COUGAR,305,315,2020,152,258,"POLYGON ((-1587662.179 2645713.160, -1590213.9..."


In [31]:
# Save out the bounds to a file for GEE.
out_fp = os.path.join(projdir, f'data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds.shp')
afd_fire_bounds_.to_file(out_fp)
print(f"Saved to {out_fp}\n")

Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds.shp



In [32]:
gc.collect()

0

In [22]:
afd_fire_bounds_.columns

NameError: name 'afd_fire_bounds_' is not defined

### Create a time-series product for GRIDMET

This needs to be a daily polygon that can be used to summarize gridmet variables for each observation. For now, taking the union of each daily FRP observation should do the trick.

In [43]:
# Create an acquisition Day of Year
afd_events_plots_c['ACQ_DOY'] = afd_events_plots_c['ACQ_DATETIME'].dt.dayofyear

# Group the AFDs by fire ID and acquisition date
daily_polygons = [] # to store the daily polygons
for (fire_id, year, doy), group in afd_events_plots_c.groupby(['NIFC_ID', 'START_YEAR', 'ACQ_DOY']):
    # Get the geometry of the group (polygons)
    polygons = group.geometry
    # Create a union of the polygons for the day
    if len(polygons) > 1:
        union_polygon = polygons.unary_union  # Union of all polygons
    else:
        union_polygon = polygons.iloc[0]  # If only one polygon, just use it
    
    # Create a GeoDataFrame row for the union polygon
    daily_polygons.append({
        'NIFC_ID': fire_id,
        'FIRE_YEAR': year,
        'ACQ_DOY': doy,
        'geometry': union_polygon
    })

# Convert the list of dictionaries to a GeoDataFrame
daily_polygons_gdf = gpd.GeoDataFrame(daily_polygons, crs=afd_events_plots_c.crs)

out_fp = os.path.join(projdir, f'data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds_daily.shp')
daily_polygons_gdf.to_file(out_fp)
print(f"Saved daily polygons to {out_fp}")

Saved daily polygons to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds_daily.shp


In [44]:
daily_polygons_gdf.head()

,NIFC_ID,FIRE_YEAR,ACQ_DOY,geometry
0,100763,2022,278,"POLYGON ((-1302924.731 2938127.779, -1302924.7..."
1,100763,2022,279,"POLYGON ((-1304119.767 2936849.317, -1304119.7..."
2,100763,2022,280,"POLYGON ((-1303310.031 2937490.782, -1303685.0..."
3,100763,2022,281,"POLYGON ((-1303467.796 2938138.289, -1303467.7..."
4,100763,2022,282,"POLYGON ((-1303703.880 2937442.765, -1303703.8..."
